In [2]:
# -------------------- Imports --------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GRU, LSTM, Conv1D, Flatten, Input, Concatenate, Dropout, Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score




In [3]:
# -------------------- Load Dataset --------------------
df = pd.read_csv("ev_charging_patterns_augmented.csv")  # Place this file in the same directory



In [4]:
# -------------------- Preprocessing --------------------
df.drop(columns=['User_ID'], inplace=True, errors='ignore')
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))
df.dropna(inplace=True)

# -------------------- Features & Targets --------------------
y1 = df['Energy Consumed (kWh)']
y2 = df['Charging Duration (hours)']
X = df.drop(['Energy Consumed (kWh)', 'Charging Duration (hours)'], axis=1)

# -------------------- Normalize Features --------------------
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# -------------------- Train/Test Split --------------------
X_train, X_test, y1_train, y1_test = train_test_split(X_scaled, y1, test_size=0.2, random_state=42)
_, _, y2_train, y2_test = train_test_split(X_scaled, y2, test_size=0.2, random_state=42)



In [5]:
# -------------------- Evaluation Function --------------------
def evaluate_model(model_name, model_func):
    model = model_func()
    history = model.fit(X_train, [y1_train, y2_train], epochs=30, batch_size=32, verbose=0, validation_split=0.1)
    preds = model.predict(X_test)
    y1_pred, y2_pred = preds[0].flatten(), preds[1].flatten()

    def metrics(true, pred):
        mae = mean_absolute_error(true, pred)
        mape = np.mean(np.abs((true - pred) / true)) * 100
        rmse = np.sqrt(mean_squared_error(true, pred))
        r2 = r2_score(true, pred)
        return mae, mape, rmse, r2

    # Plotting loss over epochs
    plt.figure(figsize=(10, 4))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f"{model_name} - Loss over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("MAE Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{model_name.replace(' ', '_')}_loss_curve.png")
    plt.close()

    return {
        'Model': model_name,
        'Energy MAE': metrics(y1_test, y1_pred)[0],
        'Energy MAPE': metrics(y1_test, y1_pred)[1],
        'Energy RMSE': metrics(y1_test, y1_pred)[2],
        'Energy R²': metrics(y1_test, y1_pred)[3],
        'Time MAE': metrics(y2_test, y2_pred)[0],
        'Time MAPE': metrics(y2_test, y2_pred)[1],
        'Time RMSE': metrics(y2_test, y2_pred)[2],
        'Time R²': metrics(y2_test, y2_pred)[3],
    }

input_shape = X_train.shape[1]



In [6]:
# -------------------- Model Architectures --------------------
def ann_gru():
    inputs = Input(shape=(input_shape,))
    x1 = Dense(64, activation='relu')(inputs)
    x1 = Dense(32, activation='relu')(x1)
    x2 = Reshape((input_shape, 1))(inputs)
    x2 = GRU(32)(x2)
    x = Concatenate()([x1, x2])
    x = Dense(64, activation='relu')(x)
    out1 = Dense(1, name='energy')(x)
    out2 = Dense(1, name='time')(x)
    model = Model(inputs, [out1, out2])
    model.compile(optimizer='adam', loss='mae')
    return model

def cnn_lstm():
    inputs = Input(shape=(input_shape,))
    x = Reshape((input_shape, 1))(inputs)
    x = Conv1D(32, 3, activation='relu')(x)
    x = LSTM(32)(x)
    x = Dense(64, activation='relu')(x)
    out1 = Dense(1, name='energy')(x)
    out2 = Dense(1, name='time')(x)
    model = Model(inputs, [out1, out2])
    model.compile(optimizer='adam', loss='mae')
    return model

def gru_lstm():
    inputs = Input(shape=(input_shape,))
    x = Reshape((input_shape, 1))(inputs)
    x = GRU(32, return_sequences=True)(x)
    x = LSTM(32)(x)
    x = Dense(64, activation='relu')(x)
    out1 = Dense(1, name='energy')(x)
    out2 = Dense(1, name='time')(x)
    model = Model(inputs, [out1, out2])
    model.compile(optimizer='adam', loss='mae')
    return model

def ann_lstm():
    inputs = Input(shape=(input_shape,))
    x1 = Dense(64, activation='relu')(inputs)
    x2 = Reshape((input_shape, 1))(inputs)
    x2 = LSTM(32)(x2)
    x = Concatenate()([x1, x2])
    x = Dense(64, activation='relu')(x)
    out1 = Dense(1, name='energy')(x)
    out2 = Dense(1, name='time')(x)
    model = Model(inputs, [out1, out2])
    model.compile(optimizer='adam', loss='mae')
    return model

def cnn_gru():
    inputs = Input(shape=(input_shape,))
    x = Reshape((input_shape, 1))(inputs)
    x = Conv1D(32, 3, activation='relu')(x)
    x = GRU(32)(x)
    x = Dense(64, activation='relu')(x)
    out1 = Dense(1, name='energy')(x)
    out2 = Dense(1, name='time')(x)
    model = Model(inputs, [out1, out2])
    model.compile(optimizer='adam', loss='mae')
    return model



In [7]:
# -------------------- Run All Models --------------------
results = []
results.append(evaluate_model("ANN + GRU", ann_gru))
results.append(evaluate_model("CNN + LSTM", cnn_lstm))
results.append(evaluate_model("GRU + LSTM", gru_lstm))
results.append(evaluate_model("ANN + LSTM", ann_lstm))
results.append(evaluate_model("CNN + GRU", cnn_gru))



56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [8]:
# -------------------- Results --------------------
results_df = pd.DataFrame(results)
pd.set_option('display.max_columns', None)
results_df = results_df.sort_values(by='Energy RMSE', ascending=True)
print("\nFinal Model Evaluation Results:")
print(results_df.round(4))

# Save to CSV
results_df.to_csv("model_comparison_results.csv", index=False)


Final Model Evaluation Results:
        Model  Energy MAE  Energy MAPE  Energy RMSE  Energy R²  Time MAE  \
0   ANN + GRU     17.3066     150.4013      21.5914     0.0211    0.8609   
2  GRU + LSTM     18.5726     169.7524      21.8304    -0.0007    0.8949   
3  ANN + LSTM     18.2922     176.4858      22.0082    -0.0171    0.8781   
1  CNN + LSTM     18.7333     180.9585      22.0980    -0.0254    0.8822   
4   CNN + GRU     18.7977     184.9187      22.2064    -0.0355    0.8814   

   Time MAPE  Time RMSE  Time R²  
0    64.1472     1.0391  -0.0064  
2    58.9569     1.0651  -0.0576  
3    69.8193     1.0494  -0.0266  
1    65.1386     1.0368  -0.0020  
4    63.7282     1.0372  -0.0028  
